In [13]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split
from spacy.util import minibatch
from spacy.training.example import Example  
import random

In [44]:

df = pd.read_csv("Tweets.csv")
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [45]:
df = df[["text", "airline_sentiment"]]
df.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [46]:
df["airline_sentiment"].unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [48]:
# Drop Neutral to a binary analysis

df = df[df["airline_sentiment"] != "neutral"]

In [49]:
df["airline_sentiment"].unique()

array(['positive', 'negative'], dtype=object)

In [50]:
df.isna().sum()

text                 0
airline_sentiment    0
dtype: int64

In [51]:
# enconding categorical field to 0 and 1 to analysis remember computer only understands numbers
df["airline_sentiment"] = df["airline_sentiment"].map({"negative":0, "positive":1})

C:\Users\Luiz_gustavo_Andrade\AppData\Local\Temp\ipykernel_32848\2257080823.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["airline_sentiment"] = df["airline_sentiment"].map({"negative":0, "positive":1})


In [52]:
df["airline_sentiment"].unique()

array([1, 0])

In [24]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/12.8 MB 1.7 MB/s eta 0:00:08
     --- ------------------------------------ 1.0/12.8 MB 1.7 MB/s eta 0:00:07
     --- ------------------------------------ 1.0/12.8 MB 1.7 MB/s eta 0:00:07
     ---- ----------------------------------- 1.3/12.8 MB 1.1 MB/s eta 0:00:11
     ---- ----------------------------------- 1.6/12.8 MB 1.1 MB/s eta 0:00:10
     ------ --------------------------------- 2.1/12.8 MB 1.3 MB/s eta 0:00:09
     ------- -------------------------------- 2.4/12.8 MB 1.3 MB/s eta 0:00:09
     -------- ------------------------------- 2.6/12.8 MB 1.3 MB/s eta 0:00:08
     --------- ------------------------------ 2.9/12.8 MB 1.3 MB/s 

In [25]:
text = "This is an example test , which describes the classification"

nlp = spacy.load("en_core_web_sm")

In [27]:
doc = nlp(text)

In [28]:
for token in doc:
    print(token.text)

This
is
an
example
test
,
which
describes
the
classification


In [30]:
ex = "organanize organizes and organization"

for token in nlp(ex):
    print(token.text, "--->", token.lemma_)

organanize ---> organanize
organizes ---> organize
and ---> and
organization ---> organization


In [31]:
# print column headers

print ('{:15} | {:15} | {:8} | {:8} | {:11} | {:8} | {:8} | {:8} | ' .format (
    'TEXT', 'LEMMA_', 'POS_', 'TAG_', 'DEP_', 'SHAPE_', 'IS_ALPHA', 'IS_STOP'))

# print various SpaCy POS attributes

for token in doc: 
    print ('{:15} | {:15} | {:8} | {:8} | {:11} | {:8} | {:8} | {:8} | ' .format (
        token.text, token.lemma_, token.pos_, token.tag_, token.dep_
        , token.shape_, token.is_alpha, token.is_stop))

TEXT            | LEMMA_          | POS_     | TAG_     | DEP_        | SHAPE_   | IS_ALPHA | IS_STOP  | 
This            | this            | PRON     | DT       | nsubj       | Xxxx     |        1 |        1 | 
is              | be              | AUX      | VBZ      | ROOT        | xx       |        1 |        1 | 
an              | an              | DET      | DT       | det         | xx       |        1 |        1 | 
example         | example         | NOUN     | NN       | compound    | xxxx     |        1 |        0 | 
test            | test            | NOUN     | NN       | attr        | xxxx     |        1 |        0 | 
,               | ,               | PUNCT    | ,        | punct       | ,        |        0 |        0 | 
which           | which           | PRON     | WDT      | nsubj       | xxxx     |        1 |        1 | 
describes       | describe        | VERB     | VBZ      | relcl       | xxxx     |        1 |        0 | 
the             | the             | DET      |

In [36]:
ner_text = "When I told John that I wanted to move to Alaska, he warned me that I'd have trouble finding a Starbucks there"
ner_doc = nlp(ner_text)

print ('{:10} | {:15}'.format('LABEL', 'ENTITY'))

for ent in ner_doc.ents[0:20]:
    print ('{:10} | {:50}'.format(ent.label_, ent.text))

LABEL      | ENTITY         
PERSON     | John                                              
GPE        | Alaska                                            
ORG        | Starbucks                                         


In [37]:

from spacy import displacy

displacy.render(docs=ner_doc, style='ent', jupyter=True)

In [ ]:
# 1. Load spaCy model

nlp = spacy.blank("en")

In [53]:
# 2. Split data 
X_train, X_test, y_train, y_test = train_test_split(df['text'], df["airline_sentiment"], test_size=0.2, random_state=42)

In [65]:
# 3. Build Text Categorizer Model

config = { 
    "threshold": 0.5,
  "model": { 
      "@architectures": "spacy.TextCatEnsemble.v2",
      "linear_model": { 
          "@architectures": "spacy.TextCatBOW.v2",
          "exclusive_classes": True,
          "ngram_size" : 1,
          "no_output_layer": False,
      },
      "tok2vec": {
          "@architectures": "spacy.Tok2Vec.v2",
          "embed": {
              "@architectures": "spacy.MultiHashEmbed.v2",
              "width": 96,
              "rows": [2000, 2000, 100, 100, 100],
               "attrs": ["NORM", "LOWER", "PREFIX", "SUFFIX", "SHAPE"],
          },
        "encode": {
            "@architectures": "spacy.MaxoutWindowEncoder.v2",
            "width": 96,
            "window_size": 4,
            "maxout_pieces": 3,
            "depth": 4,
        },
      },
  },
}

In [66]:
texcat = nlp.add_pipe("textcat", config=config)
texcat.add_label("positive")
texcat.add_label("negative")

1

In [68]:
# 4 Defining the training data 
train_data = list(zip(X_train, [{"cats": {"positive": bool(label), "negative": not bool(label)}} for label in y_train]))
random.shuffle(train_data)

In [ ]:
# define de optmizer

optimizer = nlp.begin_training()

for epoch in range(10):
    losses = {}
    random.shuffle(train_data)
    batches = minibatch(train_data, size=8)
    # create the minibatch
    for batch in batches:
        # unpack the batch
        texts, annotations = zip(*batch)
        # create the Example objects
        examples = [Example.from_dict(nlp.make_doc(text), annotation) for text, annotation in zip(texts, annotations)]
        # update the model
        nlp.update(examples, sgd=optimizer, drop=0.2, losses=losses)
    print(f"Epoch {epoch + 1}, Loss: {losses['textcat']}")

# 5. Evaluate the model

y_pred = [nlp(text).cats ["positive"] > 0.5 for text in X_test]




Epoch 1, Loss: 116.79460025883918
Epoch 2, Loss: 79.84651063578116
Epoch 3, Loss: 67.17048048218803
Epoch 4, Loss: 53.06857063270419
Epoch 5, Loss: 44.10873527674465
Epoch 6, Loss: 34.13976941165087
Epoch 7, Loss: 29.64964518223815
Epoch 8, Loss: 25.874885351473935
Epoch 9, Loss: 20.266115269780343
Epoch 10, Loss: 19.29401841390989


NameError: name 'classification_report' is not defined

In [71]:
from sklearn.metrics import classification_report
print (classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1862
           1       0.81      0.75      0.78       447

    accuracy                           0.92      2309
   macro avg       0.87      0.85      0.86      2309
weighted avg       0.91      0.92      0.92      2309

